In [1]:
import pyhepmc
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

print("All packages loaded successfully!")

All packages loaded successfully!


In [5]:
import pyhepmc
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact

# File path
hepmc_file = "data/top.hepmc"

# Define PID codes for electrons and muons
electron_pids = {11, -11}  # e⁻, e⁺
muon_pids = {13, -13}  # μ⁻, μ⁺

# Function to calculate transverse momentum (pT) and pseudorapidity (η)
def calculate_kinematics(p):
    pt = np.sqrt(p.px**2 + p.py**2)
    eta = 0.5 * np.log((p.e + p.pz) / (p.e - p.pz)) if abs(p.e - p.pz) > 0 else 0
    return pt, abs(eta)

# Load events and extract electrons/muons with status=2
def load_events(hepmc_file):
    electrons = []
    muons = []

    with pyhepmc.open(hepmc_file) as f:
        for event in f:
            for particle in event.particles:
                if particle.status == 1:  # Select only status=2 particles
                    pt, eta = calculate_kinematics(particle.momentum)

                    if particle.pid in electron_pids:
                        electrons.append((pt, eta))
                    elif particle.pid in muon_pids:
                        muons.append((pt, eta))

    return electrons, muons

# Load data
electrons, muons = load_events(hepmc_file)

# Function to plot histogram with η restriction
def plot_pt_distribution(max_eta=2.5):

    filtered_electrons = [pt for pt, eta in electrons if eta <= max_eta]
    filtered_muons = [pt for pt, eta in muons if eta <= max_eta]

    plt.figure(figsize=(8, 6))
    plt.hist(filtered_electrons, bins=50, alpha=0.6, color='blue', edgecolor='black', label="Electrons")
    plt.hist(filtered_muons, bins=50, alpha=0.6, color='red', edgecolor='black', label="Muons")

    plt.xlabel("Transverse Momentum (GeV)")
    plt.ylabel("Frequency")
    plt.title(f"Transverse Momentum Distribution (|η| ≤ {max_eta})")
    plt.yscale("log")  # Log scale for better visibility
    plt.legend()
    plt.grid(True)
    plt.show()

# Create an interactive slider for η restriction
interact(plot_pt_distribution, max_eta=widgets.FloatSlider(min=0, max=3.0, step=0.1, value=2, description="Max |η|"));

interactive(children=(FloatSlider(value=2.0, description='Max |η|', max=3.0), Output()), _dom_classes=('widget…